Лабораторная работа №4

Выполнила студентка группы 6131-010402D Фролова Анастасия Сергеевна

1:

Считать в pandas.DataFrame любой источник данных: CSV, JSON, Excel-файл, HTML-таблицу и т.п.

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from matplotlib import pyplot as plt

import sklearn as sk
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

2:

Датасет и подготовка данных:


Привести описание датасета.


In [ ]:
path = '...'

df = pd.read_csv(Path(path, 'healthcare-dataset-stroke-data.csv'))

Датасет содержит данные о пациентах. Эти данные используются для прогнозирования вероятности инсульта у пациента на основе таких входных параметров, как пол, возраст, различные заболевания и статус курения.
* id - уникальный идентификатор;
* gender - «Мужской», «Женский» или «Другой»;
* age - возраст пациента;
* hypertension - наличие или отсутствие гипертензии;
* heart_disease - наличие или отсутствие заболеваний сердца;
* ever_married - женат или нет;
* work_type - «Ребенок», «Госслужащий», «Безработный», «ИП» или «Самозанятый»;
* Residence_type - «Сельское» или «Городское»;
* avg_glucose_level - средний уровень глюкозы в крови;
* bmi - индекс массы тела;
* smoking_status - «ранее курил», «никогда не курил», «курящий» или «неизвестно»;
* stroke - наличие или отсутствие инсульта.

In [3]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


Осуществить предобработку данных (избавиться от null, убрать некоторые признаки и т.п.) - "подчистить данные".


In [4]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [5]:
# Избавляемся от NaN

df_no_NaN = df.dropna()
df_no_NaN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4909 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4909 non-null   int64  
 1   gender             4909 non-null   object 
 2   age                4909 non-null   float64
 3   hypertension       4909 non-null   int64  
 4   heart_disease      4909 non-null   int64  
 5   ever_married       4909 non-null   object 
 6   work_type          4909 non-null   object 
 7   Residence_type     4909 non-null   object 
 8   avg_glucose_level  4909 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     4909 non-null   object 
 11  stroke             4909 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 498.6+ KB


Закодировать категориальные признаки при необходимости.


In [6]:
print(df_no_NaN['gender'].unique())
print(df_no_NaN['ever_married'].unique())
print(df_no_NaN['work_type'].unique())
print(df_no_NaN['Residence_type'].unique())
print(df_no_NaN['smoking_status'].unique())

['Male' 'Female' 'Other']
['Yes' 'No']
['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
['Urban' 'Rural']
['formerly smoked' 'never smoked' 'smokes' 'Unknown']


In [ ]:
# Заменим категориальные признаки
# Unknown заменим на NaN и исключим их

columns = {'gender': {'Male': 1, 'Female': 2, 'Other': 0},
           'ever_married': {'Yes': 1, 'No': 0},
           'work_type': {'Private': 0, 'Self-employed': 1, 'Govt_job': 2, 'children': 3, 'Never_worked': 4},
           'Residence_type': {'Urban': 1, 'Rural': 0},
           'smoking_status': {'formerly smoked': 1, 'never smoked': 0, 'smokes': 2, 'Unknown': np.nan}}
df_drop = df_no_NaN.replace(columns)
df_drop = df_drop.dropna()

In [8]:
print(df_drop.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3426 entries, 0 to 5108
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3426 non-null   int64  
 1   gender             3426 non-null   int64  
 2   age                3426 non-null   float64
 3   hypertension       3426 non-null   int64  
 4   heart_disease      3426 non-null   int64  
 5   ever_married       3426 non-null   int64  
 6   work_type          3426 non-null   int64  
 7   Residence_type     3426 non-null   int64  
 8   avg_glucose_level  3426 non-null   float64
 9   bmi                3426 non-null   float64
 10  smoking_status     3426 non-null   float64
 11  stroke             3426 non-null   int64  
dtypes: float64(4), int64(8)
memory usage: 348.0 KB
None


Нормализовать данные.


Разбить выборку на обучающую и тестовую.


In [9]:
# Выполним нормализацию и разбиение

X = df_drop.drop('stroke', axis=1)
y = df_drop['stroke']

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X)
X = pd.DataFrame(scaled_features, columns=X.columns)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 42, stratify=y)

3:

Дерево решений:


С использованием GridSearchCV осуществить подбор гиперпараметра DecisionTreeClassifier (как минимум max_depth, max_features, другие параметры - по желанию.)


In [10]:
grid = {
    'max_depth': range(1, X.shape[1]),
    'max_features': range(2, 20),
    'criterion': ['gini', 'entropy', 'log_loss'],
}

model_tree = DecisionTreeClassifier()

search = GridSearchCV(model_tree, grid, cv = 5, n_jobs= -1, verbose = 2)
search.fit(x_train, y_train)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 11),
                         'max_features': range(2, 20)},
             verbose=2)

In [11]:
search.best_params_

{'criterion': 'entropy', 'max_depth': 3, 'max_features': 7}

In [12]:
best_model = search.best_estimator_
best_model_pred = best_model.predict(x_train)

In [13]:
print(f"Accuracy: {accuracy_score(y_train, best_model_pred)}; Precision: {precision_score(y_train, best_model_pred, average='weighted', zero_division=0)}; Recall: {recall_score(y_train, best_model_pred, average='weighted')}; F1_score: {f1_score(y_train, best_model_pred, average='weighted')}")

Accuracy: 0.9474562135112594; Precision: 0.8976732765210932; Recall: 0.9474562135112594; F1_score: 0.9218931550739106


In [14]:
print(f"Classification Report:")
print(classification_report(y_train, best_model_pred, zero_division=0))

Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      2272
           1       0.00      0.00      0.00       126

    accuracy                           0.95      2398
   macro avg       0.47      0.50      0.49      2398
weighted avg       0.90      0.95      0.92      2398



Осуществить фильтрацию признаков (по какому-нибудь значению порога важности признака).


In [18]:
# Выводим feature_importance и сортируем с последующей фильтрацией

print(f"Most important features:")

features = zip(x_train.columns, best_model.feature_importances_)
features = sorted(features, key=lambda x: x[1], reverse=True)
for f_name, f_imp in features:
    if f_imp >= 0.01:
        print(f"Feature {f_name} with contribution {f_imp:.3f}")

Most important features:
Feature age with contribution 0.885
Feature avg_glucose_level with contribution 0.065
Feature hypertension with contribution 0.031
Feature bmi with contribution 0.020


Подобрать лучшую модель с использованием GridSearchCV на обучающей выборке с отфильтрованными признаками.


In [ ]:
# Выбираем 1 признак, так как его важность превосходит остальные

selected_features = ['age']
x_train_filtered = x_train[selected_features]

search_filtered = GridSearchCV(model_tree, grid, cv = 5, n_jobs= -1, verbose= 2)
search_filtered.fit(x_train_filtered, y_train)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 11),
                         'max_features': range(2, 20)},
             verbose=2)

In [25]:
search_filtered.best_params_

{'criterion': 'gini', 'max_depth': 1, 'max_features': 2}

In [26]:
best_model_filter = search_filtered.best_estimator_
best_model_filter_pred = best_model_filter.predict(x_train_filtered)

In [27]:
print(f"Accuracy: {accuracy_score(y_train, best_model_filter_pred)}; Precision: {precision_score(y_train, best_model_filter_pred, average='weighted', zero_division=0)}; Recall: {recall_score(y_train, best_model_filter_pred, average='weighted')}; F1_score: {f1_score(y_train, best_model_filter_pred, average='weighted')}")

Accuracy: 0.9474562135112594; Precision: 0.8976732765210932; Recall: 0.9474562135112594; F1_score: 0.9218931550739106


In [28]:
print(f"Classification Report:")
print(classification_report(y_train, best_model_filter_pred, zero_division=0))

Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      2272
           1       0.00      0.00      0.00       126

    accuracy                           0.95      2398
   macro avg       0.47      0.50      0.49      2398
weighted avg       0.90      0.95      0.92      2398



Сравнить метрики до и после фильтрации признаков лучших моделей.


In [ ]:
# После фильтрации признаков метрики не изменились, но важно заметить, что расчеты проходят значительно быстрее, ведь мы используем лишь один признак

4:

Случайный лес


Построить случайный лес (RandomForestClassifier), c использованием GridSearchCV осуществить подбор гиперпараметра.


In [29]:
grid_rand = {
    'max_depth': range(1, X.shape[1]),
    'n_estimators': range(10, 30, 10),
    'criterion': ['gini', 'entropy', 'log_loss']
}

model_rand_tree = RandomForestClassifier()

search_rt = GridSearchCV(model_rand_tree, grid_rand, cv = 5, n_jobs= -1, verbose = 2)
search_rt.fit(x_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 11),
                         'n_estimators': range(10, 30, 10)},
             verbose=2)

In [30]:
search_rt.best_params_

{'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 20}

In [31]:
best_model_rt = search_rt.best_estimator_
best_model_pred_rt = best_model_rt.predict(x_train)

In [32]:
print(f"Accuracy: {accuracy_score(y_train, best_model_pred)}; Precision: {precision_score(y_train, best_model_pred_rt, average='weighted', zero_division=0)}; Recall: {recall_score(y_train, best_model_pred_rt, average='weighted')}; F1_score: {f1_score(y_train, best_model_pred_rt, average='weighted')}")

Accuracy: 0.9474562135112594; Precision: 0.9604394073649876; Recall: 0.9587155963302753; F1_score: 0.945799019210717


Осуществить фильтрацию признаков.


In [33]:
# Выводим feature_importance и сортируем с последующей фильтрацией

print(f"Most important features:")

features_rt = zip(x_train.columns, best_model_rt.feature_importances_)
features_rt = sorted(features_rt, key=lambda x: x[1], reverse=True)
for f_name, f_imp in features_rt:
    if f_imp >= 0.1:
        print(f"Feature {f_name} with contribution {f_imp}")

Most important features:
Feature age with contribution 0.31258103225452916
Feature avg_glucose_level with contribution 0.19671876734293312
Feature id with contribution 0.14955841593414088
Feature bmi with contribution 0.13087138928172953


Подобрать лучшую модель с использованием GridSearchCV на обучающей выборке с отфильтрованными признаками.


In [35]:
# Выбираем 4 первых признака

selected_features_rt = ['age', 'avg_glucose_level', 'id', 'bmi']
x_train_filtered_rt = x_train[selected_features_rt]

search_filtered_rt = GridSearchCV(model_rand_tree, grid_rand, cv = 5, n_jobs= -1, verbose= 2)
search_filtered_rt.fit(x_train_filtered_rt, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 11),
                         'n_estimators': range(10, 30, 10)},
             verbose=2)

In [36]:
search_filtered_rt.best_params_

{'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 10}

In [38]:
best_model_filter_rt = search_filtered_rt.best_estimator_
best_model_filter_pred_rt = best_model_filter_rt.predict(x_train_filtered_rt)

In [39]:
print(f"Accuracy: {accuracy_score(y_train, best_model_filter_pred_rt)}; Precision: {precision_score(y_train, best_model_filter_pred_rt, average='weighted', zero_division=0)}; Recall: {recall_score(y_train, best_model_filter_pred_rt, average='weighted')}; F1_score: {f1_score(y_train, best_model_filter_pred_rt, average='weighted')}")

Accuracy: 0.9495412844036697; Precision: 0.9520926862369986; Recall: 0.9495412844036697; F1_score: 0.9268922266991745


Сравнить метрики до и после фильтрации признаков лучших моделей.


In [ ]:
# Точность очень незначительно выросла (разница в тысячных и десятитысячных), однако остальные метрики ухудшились, что говорит о том, что в результате фильтрации и дальнейшего исключения признаков была сокращена важная информация для классификации

5:

Метод ближайших соседей:


С использованием GridSearchCV осуществить подбор гиперпараметра KNeighborsClassifier (n_neighbors).


In [40]:
grid_knn = {
    'n_neighbors': range(2, 20)
}

model_knn = KNeighborsClassifier()

search_knn = GridSearchCV(model_knn, grid_knn, cv = 5, n_jobs= -1, verbose = 2)
search_knn.fit(x_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(2, 20)}, verbose=2)

In [41]:
search_knn.best_params_

{'n_neighbors': 4}

In [42]:
best_model_knn = search_knn.best_estimator_
best_model_pred_knn = best_model_rt.predict(x_train)

In [43]:
print(f"Accuracy: {accuracy_score(y_train, best_model_pred_knn)}; Precision: {precision_score(y_train, best_model_pred_knn, average='weighted', zero_division=0)}; Recall: {recall_score(y_train, best_model_pred_knn, average='weighted')}; F1_score: {f1_score(y_train, best_model_pred_knn, average='weighted')}")

Accuracy: 0.9587155963302753; Precision: 0.9604394073649876; Recall: 0.9587155963302753; F1_score: 0.945799019210717


Осуществить фильтрацию признаков.


In [ ]:
# Для метода ближайших соседей нет feature_importances_, поэтому выберем из предыдущих моделей набор отфильтрованных признаков (был выбран набор из дерева решений)

x_train_filtered_knn = x_train_filtered_rt
x_train_filtered_knn.head()

,age,avg_glucose_level,id,bmi
1273,0.708333,0.643569,0.658854,0.416149
835,0.958333,0.061306,0.015200,0.131677
1020,0.430556,0.116148,0.156019,0.320497
2406,0.847222,0.682578,0.304650,0.370186
3029,1.000000,0.011264,0.859950,0.198758


Подобрать лучшую модель с использованием GridSearchCV на обучающей выборке с отфильтрованными признаками.


In [51]:
search_filtered_knn = GridSearchCV(model_knn, grid_knn, cv = 5, n_jobs= -1, verbose= 2)
search_filtered_knn.fit(x_train_filtered_knn, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(2, 20)}, verbose=2)

In [52]:
search_filtered_knn.best_params_

{'n_neighbors': 12}

In [53]:
best_model_filter_knn = search_filtered_knn.best_estimator_
best_model_filter_pred_knn = best_model_filter_knn.predict(x_train_filtered_knn)

In [49]:
print(f"Accuracy: {accuracy_score(y_train, best_model_filter_pred_knn)}; Precision: {precision_score(y_train, best_model_filter_pred_knn, average='weighted', zero_division=0)}; Recall: {recall_score(y_train, best_model_filter_pred_knn, average='weighted')}; F1_score: {f1_score(y_train, best_model_filter_pred_knn, average='weighted')}")

Accuracy: 0.9474562135112594; Precision: 0.8976732765210932; Recall: 0.9474562135112594; F1_score: 0.9218931550739106


Сравнить метрики до и после фильтрации признаков.


In [ ]:
# В данном случае использование меньше набора признаков привело к ухудшению результатов точности и метрик precision, recall, f1

6:

Если наблюдается улучшение метрик после фильтрации признаков хотя бы для одной из моделей, то для набора отфильтрованных признаков (пересечение множеств отфильтрованных признаков каждой модели или объединение множеств – не особо важно, главное описать, каким образом получен новый subset данных) заново построить наилучшие модели KNeighborsClassifier, DecisionTreeClassifier, RandomForestClassifier, сравнить модели в пункте 7 на одинаковом полученном наборе отфильтрованных признаков. Иначе – пропустить этот пункт.


In [ ]:
# Улучшения метрик после фитральции не наблюдалось, была лишь незначительная разница в точности для модели случайного леса, однако также присутствовало ухудшение метрик precision, recall, f1
# В случае остальных методов классификации значения метрик лишь уменьшались

7:

Оценка качества построенных моделей:


In [61]:

from sklearn.tree import export_graphviz

Визуализировать любое полученное дерево решений.


In [ ]:
# Вписать свой путь

data_path = "..."

# Используем .dot формат для визуализации дерева
export_graphviz(
    best_model,
    feature_names=x_train.columns,
    class_names=['Stroke','No stroke'],
    out_file = f'{data_path}/small_tree.dot',
    filled = True
)

!dot -Tpng ./images/small_tree.dot -o ./images/small_tree.png

Сравнить лучшие модели KNeighborsClassifier, DecisionTreeClassifier, RandomForestClassifier на тестовой выборке. Привести значения метрик 


In [71]:
print("DecisionTreeClassifier:")
print(f"Accuracy: {accuracy_score(y_test, best_model.predict(x_test)):.3f}")
print(f"Precision: {precision_score(y_test, best_model.predict(x_test), average='weighted', zero_division=0):.3f}")
print(f"Recall: {recall_score(y_test, best_model.predict(x_test), average='weighted', zero_division=0):.3f}")
print(f"F1_score: {f1_score(y_test, best_model.predict(x_test), average='weighted', zero_division=0):.3f}")
print("RandomForestClassifier:")
print(f"Accuracy: {accuracy_score(y_test, best_model_rt.predict(x_test)):.3f}")
print(f"Precision: {precision_score(y_test, best_model_rt.predict(x_test), average='weighted', zero_division=0):.3f}")
print(f"Recall: {recall_score(y_test, best_model_rt.predict(x_test), average='weighted', zero_division=0):.3f}")
print(f"F1_score: {f1_score(y_test, best_model_rt.predict(x_test), average='weighted', zero_division=0):.3f}")
print("KNeighborsClassifier:")
print(f"Accuracy: {accuracy_score(y_test, best_model_knn.predict(x_test)):.3f}")
print(f"Precision: {precision_score(y_test, best_model_knn.predict(x_test), average='weighted', zero_division=0):.3f}")
print(f"Recall: {recall_score(y_test, best_model_knn.predict(x_test), average='weighted', zero_division=0):.3f}")
print(f"F1_score: {f1_score(y_test, best_model_knn.predict(x_test), average='weighted', zero_division=0):.3f}")

DecisionTreeClassifier:
Accuracy: 0.947
Precision: 0.898
Recall: 0.947
F1_score: 0.922
RandomForestClassifier:
Accuracy: 0.947
Precision: 0.898
Recall: 0.947
F1_score: 0.922
KNeighborsClassifier:
Accuracy: 0.946
Precision: 0.916
Recall: 0.946
F1_score: 0.923


In [ ]:
# Лучший результат дает метод ближайших соседей, если нам важны значения метриц, разница в точности моделей составляет 0.001. Если нам важна именно точность, то лучше деревья решений или случайный лес